# Progetto esame parte 1

## _Import dei dati_

In [17]:
import pandas as pd
dati = pd.read_csv("ensembl.csv", delimiter=',')
dati.head()

,seqname,source,feature,start,end,score,strand,frame,hid,hstart,hend,genscan,gene_id,transcript_id,exon_id,gene_type,variation_name,probe_name
0,8,Ensembl,similarity,26291624,26291626,904,+,.,H2RM13.1,17.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8,Ensembl,similarity,26291709,26291825,1322,+,.,L8I4P1.1,4.0,42.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8,Ensembl,similarity,26291709,26291825,904,+,.,H2RM13.1,19.0,57.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8,Ensembl,similarity,26291709,26291825,1329,+,.,D2GZ75.1,24.0,62.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,Ensembl,similarity,26291709,26291825,887,+,.,T1P9P9.1,13.0,49.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## _1. Estrarre le sole righe con gene_id uguale a ENSG00000221914.11_

In [18]:
ens = dati[dati['gene_id'] == 'ENSG00000221914.11']
ens.head()

,seqname,source,feature,start,end,score,strand,frame,hid,hstart,hend,genscan,gene_id,transcript_id,exon_id,gene_type,variation_name,probe_name
21846,8,Ensembl,gene,26291508,26291826,.,+,-1,NaN,NaN,NaN,NaN,ENSG00000221914.11,ENST00000380737.8,ENSE00002131643,protein_coding,NaN,NaN
21847,8,Ensembl,gene,26293666,26293740,.,+,1,NaN,NaN,NaN,NaN,ENSG00000221914.11,ENST00000380737.8,ENSE00003521886,protein_coding,NaN,NaN
21848,8,Ensembl,gene,26338890,26338987,.,+,1,NaN,NaN,NaN,NaN,ENSG00000221914.11,ENST00000380737.8,ENSE00003472689,protein_coding,NaN,NaN
21849,8,Ensembl,gene,26354468,26354633,.,+,0,NaN,NaN,NaN,NaN,ENSG00000221914.11,ENST00000380737.8,ENSE00003506789,protein_coding,NaN,NaN
21850,8,Ensembl,gene,26360169,26360281,.,+,1,NaN,NaN,NaN,NaN,ENSG00000221914.11,ENST00000380737.8,ENSE00003484306,protein_coding,NaN,NaN


## _2. Per ogni esone (exon_id non mancante), costruire la sequenza di nucleotidi, sfruttando le coordinate start ed end_

In [19]:
esoni = ens[ens['exon_id'] != "NaN"]
esoni.head()
print(len(ens))
print(len(esoni))

177
177


Non risultano esoni con exon_id mancante. Sono mantenuti in analisi i 177 esoni iniziali con gene_id uguale a ENSG00000221914.11

## _Lettura file gene.txt_

In [20]:
with open("gene.txt") as file:
    righe = file.readlines()
lista = []
for line in righe:
    line.rstrip()
    if line[0] == '>':
        diz = {'header':line[1:].strip('\n'), 'sequence':''}
        lista.append(diz)
    else:
        seq = line.strip('\n')
        diz['sequence'] += seq
    
print(lista)

[{'header': '8 dna:chromosome chromosome:GRCh38:8:26291508:26372680:1', 'sequence': '...'}]


### _HEADER - da header si ricavano start, end del genoma_


In [21]:
header = diz["header"].split(":")
sequence = diz["sequence"]
genoma_start = int(header[4]) #26291508
genoma_end = int(header[5])   #26372680

### _Si costruisce la colonna 'pos' e la colonna 'delta'_

In [22]:
esoni['delta'] = esoni["end"] - esoni["start"]
esoni['pos'] = esoni['start'] - genoma_start
print(esoni.head())

       seqname   source feature     start       end score strand frame  hid  \
21846        8  Ensembl    gene  26291508  26291826     .      +    -1  NaN   
21847        8  Ensembl    gene  26293666  26293740     .      +     1  NaN   
21848        8  Ensembl    gene  26338890  26338987     .      +     1  NaN   
21849        8  Ensembl    gene  26354468  26354633     .      +     0  NaN   
21850        8  Ensembl    gene  26360169  26360281     .      +     1  NaN   

       hstart  hend genscan             gene_id      transcript_id  \
21846     NaN   NaN     NaN  ENSG00000221914.11  ENST00000380737.8   
21847     NaN   NaN     NaN  ENSG00000221914.11  ENST00000380737.8   
21848     NaN   NaN     NaN  ENSG00000221914.11  ENST00000380737.8   
21849     NaN   NaN     NaN  ENSG00000221914.11  ENST00000380737.8   
21850     NaN   NaN     NaN  ENSG00000221914.11  ENST00000380737.8   

               exon_id       gene_type variation_name  probe_name  delta  \
21846  ENSE00002131643  prot

### Si inserisce nel DataFrame la sequenza di nucleotidi associata ad ogni esone 

In [23]:
esoni['seq'] = esoni.apply(lambda x: sequence[x['pos']:(x['pos']+ x['delta'])],axis=1)
print(esoni.head())

       seqname   source feature     start       end score strand frame  hid  \
21846        8  Ensembl    gene  26291508  26291826     .      +    -1  NaN   
21847        8  Ensembl    gene  26293666  26293740     .      +     1  NaN   
21848        8  Ensembl    gene  26338890  26338987     .      +     1  NaN   
21849        8  Ensembl    gene  26354468  26354633     .      +     0  NaN   
21850        8  Ensembl    gene  26360169  26360281     .      +     1  NaN   

       hstart  ...  genscan             gene_id      transcript_id  \
21846     NaN  ...      NaN  ENSG00000221914.11  ENST00000380737.8   
21847     NaN  ...      NaN  ENSG00000221914.11  ENST00000380737.8   
21848     NaN  ...      NaN  ENSG00000221914.11  ENST00000380737.8   
21849     NaN  ...      NaN  ENSG00000221914.11  ENST00000380737.8   
21850     NaN  ...      NaN  ENSG00000221914.11  ENST00000380737.8   

               exon_id       gene_type variation_name probe_name  delta  \
21846  ENSE00002131643  prote

In [24]:
# Primo tentativo di risoluzione
esoni['seq'] = 0
for i in esoni.index:
    esoni['seq'].loc[i] = sequence[esoni['pos'][i]:int(esoni['pos'][i] + esoni['delta'][i])]
print(esoni['seq'].head())

21846    AGGCCAGCCGGCGCCATTTTGAAAGTGGAGTCGCCTGCCCCTGCCG...
21847    GAGCTGGAGGAGGGAATGATATTCAGTGGTGTTTTTCTCAGGTGAA...
21848    CAGATATAATTTCTACAGTAGAATTTAATCATTCTGGAGAATTACT...
21849    AACAAAATCCAGTCTCATAGCAGAGGAGAATATAATGTTTACAGCA...
21850    ATAAAACAATAAAATTATGGAAAATCAGTGAAAGGGACAAAAGACC...
Name: seq, dtype: object


/Users/saret/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


## _3. Per ogni trascritto, determinare la somma delle lunghezze dei suoi esoni._

Si costruisce una serie per mantenere in memoria la somma delle lunghezze. Se non si facesse, sarebbe necessario ripeterlo per l'individuazione del massimo.

In [25]:
x = esoni.groupby(["transcript_id"]).sum(["delta"])["delta"]
print(x)

transcript_id
ENST00000315985.7    1868
ENST00000380737.8    3913
ENST00000517754.1    2353
ENST00000518208.1    1488
ENST00000518215.5    1156
ENST00000518254.5     751
ENST00000518397.5     823
ENST00000518890.5    1232
ENST00000519439.5     653
ENST00000519636.5     569
ENST00000520329.5     830
ENST00000521095.5     575
ENST00000521484.5     551
ENST00000521557.5     595
ENST00000522150.5     569
ENST00000522482.5     571
ENST00000522535.5     647
ENST00000523473.5     649
ENST00000523925.5     523
ENST00000523964.5     561
ENST00000524099.5     564
ENST00000524169.5     778
ENST00000657943.1    1809
ENST00000660435.1    2117
ENST00000665751.1    1695
ENST00000665949.1    2141
ENST00000666129.1    2306
ENST00000667651.1    2024
Name: delta, dtype: int64


## _4. Determinare il trascritto che ha massima lunghezza calcolata al punto precedente._

In [26]:
print(x.max(), x.idxmax())

#print(x.max(), esoni.loc[esoni["delta"].idxmax()]) #21855

3913 ENST00000380737.8
